### Calling API to get job details

In [ ]:
import requests
import pandas as pd

# Define roles and constants
roles = ["ML Engineer ",
"Cybersecurity engineer",
"QA Engineer",
"Data Scientist",
"Data Analyst",
"Software Developer",
"Web Developer"]

api_key = "123"  # Replace with your actual API key
base_url = f"https://serpapi.com/search?engine=google_jobs&hl=en&api_key={api_key}"
calls_per_role = 25

def fetch_jobs_for_role(role):
    """
    Fetch jobs for a given role, making up to `calls_per_role` API calls using pagination.
    """
    all_jobs = []
    current_url = f"{base_url}&q={role.replace(' ', '+')}"

    for call in range(calls_per_role):
        print(f"Fetching page {call + 1} for role: {role}")
        current_url = current_url + f"&api_key={api_key}"
        response = requests.get(current_url)

        if response.status_code != 200:
            print(f"Error: {response.status_code} for {current_url}")
            break

        data = response.json()

        # Extract job results
        if "jobs_results" in data:
            for job in data["jobs_results"]:
                # Add role to the job data
                job["role"] = role
                all_jobs.append(job)
        else:
            print("No jobs found in response.")
            break

        # Get the next page URL
        pagination = data.get("serpapi_pagination", {})
        next_url = pagination.get("next")
        if not next_url:
            print("No more pages available.")
            break

        current_url = next_url  # Update the URL for the next call

    return all_jobs

def main():
    all_roles_jobs = []

    for role in roles:
        role_jobs = fetch_jobs_for_role(role)
        all_roles_jobs.extend(role_jobs)

    # Convert to a pandas DataFrame
    jobs_df = pd.DataFrame(all_roles_jobs)
    print(f"Final DataFrame contains {len(jobs_df)} job postings.")

    # Save the DataFrame to a CSV file
    jobs_df.to_csv("/content/drive/MyDrive/Data_Management/all_jobs.csv", index=False)
    print("Saved all jobs")

    return jobs_df

if __name__ == "__main__":
    final_df = main()


Fetching page 1 for role: Web Developer
Fetching page 2 for role: Web Developer
Fetching page 3 for role: Web Developer
Fetching page 4 for role: Web Developer
Fetching page 5 for role: Web Developer
Fetching page 6 for role: Web Developer
Fetching page 7 for role: Web Developer
Fetching page 8 for role: Web Developer
Fetching page 9 for role: Web Developer
Fetching page 10 for role: Web Developer
Fetching page 11 for role: Web Developer
Fetching page 12 for role: Web Developer
Fetching page 13 for role: Web Developer
Fetching page 14 for role: Web Developer
No jobs found in response.
Final DataFrame contains 128 job postings.
Saved all jobs


### Making one final CSV file

In [ ]:
import glob

path = '/content/drive/MyDrive/Data_Management'
file_paths = glob.glob(path + "/*.csv")

In [ ]:
def concatenate_csv_files(file_paths):
    # Initialize an empty list to store DataFrames
    dataframes = []

    for file_path in file_paths:
        # Read each file and append it to the list
        df = pd.read_csv(file_path)
        dataframes.append(df)

    # Concatenate all DataFrames, aligning columns by name
    final_df = pd.concat(dataframes, ignore_index=True, sort=False)
    return final_df

# Concatenate all CSVs into one DataFrame
final_dataframe = concatenate_csv_files(file_paths)

# Save the final DataFrame to a new CSV file
final_dataframe.to_csv('/content/drive/MyDrive/Data_Management/final_all_jobs.csv', index=False)

# Display basic information about the final DataFrame
print(f"Final DataFrame shape: {final_dataframe.shape}")
print("Saved as: /content/drive/MyDrive/Data_Management/final_all_jobs.csv")

Final DataFrame shape: (1959, 21)
Saved as: /content/drive/MyDrive/Data_Management/final_all_jobs.csv


In [ ]:
df =df[['title', 'company_name', 'location', 'via', 'share_link', 'thumbnail',
       'extensions', 'detected_extensions', 'job_highlights',
       'apply_options', 'job_id', 'role']]
df.to_csv('/content/drive/MyDrive/Data_Management/final_all_jobs.csv', index=False)


### Data pushing to big query table

In [ ]:
!pip install google-cloud-bigquery pandas

In [ ]:
!gcloud auth application-default login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=Fony3duEuHDj1D9No0ekAI2oFwROEb&prompt=consent&token_usage=remote&access_type=offline&code_challenge=63Thw0GcwDjI-iYCJpY1WbgnhVd4zbl_Lz5lgbmxnDM&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AeanS0Z4QP93iaR2JJ5UkjV2HXixvx5kuZ0_rwQnPR7RhJsShuvEUk_yTlvdyiYMQ4_Ppg

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

In [ ]:
from google.cloud import bigquery
import pandas as pd

# Define variables
project_id = "gcp-services-442706"
dataset_id = "data_management"
table_id = "jobs"

# Load CSV into a Pandas DataFrame
df = pd.read_csv('/content/drive/MyDrive/Data_Management/final_all_jobs.csv')
df.shape

(653, 12)

In [ ]:
df.columns

Index(['title', 'company_name', 'location', 'via', 'share_link', 'thumbnail',
       'extensions', 'detected_extensions', 'job_highlights', 'apply_options',
       'job_id', 'role'],
      dtype='object')

In [ ]:
client = bigquery.Client(project=project_id)

# Define BigQuery table reference
table_ref = f"{project_id}.{dataset_id}.{table_id}"

# Define job configuration
job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,  # Overwrite table; change to WRITE_APPEND if needed
    source_format=bigquery.SourceFormat.CSV,  # Specify CSV as source
    autodetect=True,  # Auto-detect schema based on CSV
)

# Load data to BigQuery
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)

# Wait for the job to complete
job.result()

# Verify upload
table = client.get_table(table_ref)  # Get the table information
print(f"Loaded {table.num_rows} rows and {len(table.schema)} columns to {table_ref}")

/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Loaded 653 rows and 12 columns to gcp-services-442706.data_management.jobs


### Data Transformation and expansion of columns

> Add blockquote



In [ ]:
import pandas as pd
import ast

# Assuming the data is already loaded into a DataFrame called `df`

# Sample representation of loading the dataset
df = pd.read_csv('/content/drive/MyDrive/Data_Management/final_all_jobs.csv')


In [ ]:
def expand_dict_column(df, column_name):
    # Convert stringified dictionaries into Python dictionaries
    df[column_name] = df[column_name].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

    # Normalize the dictionary column into separate columns
    expanded_df = pd.json_normalize(df[column_name])

    # Combine the original DataFrame with the expanded DataFrame
    df = pd.concat([df.drop(columns=[column_name]), expanded_df], axis=1)
    return df

# Expand the 'detected_extensions' column
df = expand_dict_column(df, "detected_extensions")

# Display the final DataFrame
# print(df)

# Save to a CSV file (optional)
df.to_csv('/content/drive/MyDrive/Data_Management/final_all_jobs_transformed.csv', index=False)

In [ ]:
df.head(2)

,title,company_name,location,via,share_link,thumbnail,extensions,job_highlights,apply_options,job_id,role,schedule_type,health_insurance,posted_at,salary,paid_time_off,dental_coverage,work_from_home,qualifications
0,Data Scientist,Wipro Limited,"Tampa, FL",Wipro Careers,https://www.google.com/search?ibp=htl;jobs&q=D...,https://serpapi.com/searches/6743c6b95ed3cab06...,"['Full-time', 'Health insurance']","[{'title': 'Qualifications', 'items': ['Work A...","[{'title': 'Wipro Careers', 'link': 'https://c...",eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCIsImNvbX...,Data Scientist,Full-time,True,NaN,NaN,NaN,NaN,NaN,NaN
1,Health Services Data Scientist,Manatee County Government,"Bradenton, FL",Indeed,https://www.google.com/search?ibp=htl;jobs&q=D...,https://serpapi.com/searches/6743c6b95ed3cab06...,"['4 days ago', '54,709.20–82,009.20 a year', '...","[{'title': 'Qualifications', 'items': ['Liftin...","[{'title': 'Indeed', 'link': 'https://www.inde...",eyJqb2JfdGl0bGUiOiJIZWFsdGggU2VydmljZXMgRGF0YS...,Data Scientist,Full-time,NaN,4 days ago,"54,709.20–82,009.20 a year",NaN,NaN,NaN,NaN


### New Data pushing to big query table

In [ ]:
from google.cloud import bigquery
import pandas as pd

# Define variables
project_id = "gcp-services-442706"
dataset_id = "data_management"
table_id = "jobs-transformed"

# Load CSV into a Pandas DataFrame
df = pd.read_csv('/content/drive/MyDrive/Data_Management/final_all_jobs_transformed.csv')
df.shape

(653, 19)

In [ ]:
client = bigquery.Client(project=project_id)

# Define BigQuery table reference
table_ref = f"{project_id}.{dataset_id}.{table_id}"

# Define job configuration
job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,  # Overwrite table; change to WRITE_APPEND if needed
    source_format=bigquery.SourceFormat.CSV,  # Specify CSV as source
    autodetect=True,  # Auto-detect schema based on CSV
)

# Load data to BigQuery
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)

# Wait for the job to complete
job.result()

# Verify upload
table = client.get_table(table_ref)  # Get the table information
print(f"Loaded {table.num_rows} rows and {len(table.schema)} columns to {table_ref}")

/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Loaded 653 rows and 19 columns to gcp-services-442706.data_management.jobs-transformed
